In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
signal = pd.read_csv(PATH["feature_x_sample.signal.tsv"], sep="\t", index_col=0)

signal.columns.name = SETTING["sample_alias"]

signal.fillna(value=0, inplace=True)

In [ ]:
k_nmfcc = ccal.mf_consensus_cluster_with_ks(
    signal,
    SETTING["nmf_ks"],
    mf_function="mf_by_multiplicative_update",
    n_job=SETTING["n_job"],
    n_clustering=SETTING["nmf_n_clustering"],
    plot_df=signal.size < SETTING["plot_heat_map_max_size"],
    plot_w=signal.shape[0] < SETTING["plot_cluster_max_size"],
    plot_h=signal.shape[1] < SETTING["plot_cluster_max_size"],
    directory_path=PATH["nmf/"],
)

In [ ]:
for k in SETTING["nmf_ks"]:

    for w_or_h, axis in (("w", 0), ("h", 1)):

        df = pd.read_csv(
            "{}/{}/{}.tsv".format(PATH["nmf/"], k, w_or_h), sep="\t", index_col=0
        )

        ccal.make_comparison_panel(
            df,
            df,
            axis=axis,
            title="NMF K={} {}".format(k, w_or_h.title()),
            name_0="Factor",
            name_1="Factor",
        )